# Задача:
    
### *У нас появился запрос из отдела продаж и маркетинга.*
### Как вы знаете «МегаФон» предлагает обширный набор различных услуг своим абонентам.
### При этом разным пользователям интересны разные услуги.
### **Поэтому необходимо построить  алгоритм, который для каждой пары пользователь-услуга определит вероятность подключения услуги.**


## Данные:
*В качестве исходных данных вам будет доступна информация об отклике абонентов на предложение подключения одной из услуг. Каждому пользователю может быть сделано несколько предложений в разное время, каждое из которых он может или принять, или отклонить.*

*Отдельным набором данных будет являться нормализованный анонимизированный набор признаков, характеризующий профиль потребления абонента. Эти данные привязаны к определенному времени, поскольку профиль абонента может меняться с течением времени.*

*Данные ***train*** и ***test*** разбиты по периодам – на ***train доступно 4 месяцев***, а на ***test отложен последующий месяц***.* 



### Итого, в качестве входных данных будут представлены:
>* ### ***data_train.csv:***
    >>* id,
    >>* vas_id,
    >>* buy_time,
    >>* target
>* ### ***features.csv.zip:***
    >>* id,
    >>* \<feature_list\> 


### И тестовый набор:

>* ### ***data_test.csv:***
    >>* id,
    >>* vas_id,
    >>* buy_time
    
>* ### target - целевая переменная, где
    >>* 1 - означает подключение услуги,
    >>* 0 - абонент не подключил услугу соответственно. 
>* ### buy_time - время покупки,
    >>* представлено в формате timestamp, для работы с этим столбцом понадобится функция ***datetime.fromtimestamp*** из модуля ***datetime***.
>* ### id - идентификатор абонента
>* ### vas_id - подключаемая услуга

*Примечание: Размер файла features.csv в распакованном виде весит 20 гб, для работы  с ним можно воспользоваться pandas.read_csv, либо можно воспользоваться библиотекой Dask.*


## Метрика
*Скоринг будет осуществляться функцией ***f1***, невзвешенным образом,
как например делает функция 
***sklearn.metrics.f1_score(…, average=’macro’)***.*

[sklearn.metrics.f1_score — scikit-learn 0.22.1 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score)


## Формат представления результата
>1. Работающая модель в формате ***pickle***, которая принимает файл ***data_test.csv*** из корневой папки и записывает в эту же папку файл ***answers_test.csv***. В этом файле должны находится 4 столбца:
    >>* buy_time,
    >>* id,
    >>* vas_id
    >>* target.  
    >>>***Target можно записать как вероятность подключения услуги***.
>2.	Код модели можно представить в виде ***jupyter-ноутбука***. 
>3.	Презентация в формате ***.pdf***, в которой необходимо отразить:
>>* Информация о модели, ее параметрах, особенностях и основных результатах.
>>* Обоснование выбора модели и ее сравнение с альтернативами.
>>* Принцип составления индивидуальных предложений для выбранных абонентов.

*Рекомендуемое количество слайдов: ***5 – 10***.*

Файл ***answers_test.csv*** с результатами работы модели, презентацию, ноутбуки и резюме необходимо прикрепить ко второму уроку “курсовой проект”.


### Импорт библиотек

In [1]:
import pandas as pd
import dill
import dask.dataframe as dd
from datetime import datetime
from dask.distributed import Client, progress

### Запуск "клиента" Dask
#### *Client() предоствит панель мониторинга [Dashboard](http://127.0.0.1:8787/status), которая полезна для получения информации о вычислениях.*


In [2]:
%%time
client = Client()
client

Wall time: 1.39 s


Client Scheduler: tcp://127.0.0.1:61404 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 15.94 GiB


In [3]:
PATH_D = './data/'

In [4]:
%%time
dd_features = dd.read_csv(f'{PATH_D}features.csv',sep='\t')
dd_train = dd.read_csv(f'{PATH_D}data_train.csv')
dd_test = dd.read_csv(f'{PATH_D}data_test.csv')

Wall time: 79.2 ms


In [5]:
def print_shape_data(dd_data):
    print(f"Shape: columns: {len(dd_data.columns)} Items: {len(dd_data.index)}")
    
def get_replace_dict(col_ending):
    """Функция возврата словаря для замены columns['Unnamed: 0', 'buy_time'] 
    через функцию .rename(columns=dict(...))"""
    return {'Unnamed: 0': f'index_{col_ending}', 'buy_time': f'buy_time_{col_ending}'}

In [6]:
%%time
print_shape_data(dd_train)

Shape: columns: 5 Items: 831653
Wall time: 1.07 s


In [7]:
%%time
print_shape_data(dd_features)

Shape: columns: 256 Items: 1524953
Wall time: 36.8 s


In [8]:
%%time
print_shape_data(dd_test)

Shape: columns: 4 Items: 71231
Wall time: 67 ms


In [9]:
%%time
dd_train.head(3)

Wall time: 450 ms


,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0


In [10]:
dd_train.tail(3)

,Unnamed: 0,id,vas_id,buy_time,target
831650,831650,158236,2.0,1546203600,0.0
831651,831651,1825525,2.0,1546203600,0.0
831652,831652,3022610,2.0,1546203600,0.0


In [11]:
%%time
dd_features.head(3)

Wall time: 933 ms


,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
0,0,2013026,1531688400,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,1,2014722,1539550800,36.690029,152.400888,448.069214,563.833202,463.841754,568.99821,-16.08618,...,-891.373846,-544.770792,-20.996269,48.369552,80.252276,-13.832889,-0.694428,-1.175933,-0.45614,0.0
2,2,2015199,1545598800,-67.019971,157.050888,-63.180786,178.103202,-68.598246,156.99821,3.51382,...,-977.373846,-613.770792,-12.996269,-37.630448,10829.252276,-25.832889,-0.694428,-12.175933,-0.45614,0.0


In [12]:
dd_features.tail(3)

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
11939,1524950,4294768,1546203600,-96.799971,-44.619112,-110.740786,-97.226798,-116.158246,-118.33179,-16.08618,...,-906.373846,-544.770792,-25.996269,34.369552,-293.747724,-12.832889,-0.694428,-12.175933,-0.45614,0.0
11940,1524951,4294877,1531688400,-96.799971,-407.329112,-110.740786,-453.586798,-116.158246,-474.69179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-305.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
11941,1524952,4294946,1535317200,-96.799971,-26.819112,-68.370786,-37.056798,-73.788246,-58.16179,-16.08618,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
%%time
dd_test.head(3)

Wall time: 66.7 ms


,Unnamed: 0,id,vas_id,buy_time
0,0,3130519,2.0,1548018000
1,1,2000860,4.0,1548018000
2,2,1099444,2.0,1546808400


In [14]:
dd_test.tail(3)

,Unnamed: 0,id,vas_id,buy_time
71228,71228,1891350,2.0,1548018000
71229,71229,2437172,2.0,1548018000
71230,71230,988236,2.0,1548018000


#### Из данных видно что "Unnamed: 0" является индексом -> "index"
#### Так же для отличия колонок из датасета признаков и датасета train и test при обединении данных нужно переименовать колонки 
Для dd_features:

    "buy_time" -> "buy_time_fea"
    "index" -> "index_fea"

Для dd_train и dd_test :
    
    "buy_time" -> "buy_time_sub"
    "index" -> "index_sub"
(feature -> fea, subscriber -> sub)


In [15]:
%%time
dd_features.buy_time = dd_features.buy_time.apply(lambda row: datetime.fromtimestamp(row))
dd_train.buy_time = dd_train.buy_time.apply(lambda row: datetime.fromtimestamp(row))
dd_test.buy_time = dd_test.buy_time.apply(lambda row: datetime.fromtimestamp(row))

Wall time: 46.8 ms


D:\programming\anaconda3\lib\site-packages\dask\dataframe\core.py:3519: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('buy_time', 'datetime64[ns]'))

  warnings.warn(meta_warning(meta))


In [16]:
%%time
dd_features = dd_features.rename(columns=get_replace_dict('fea'))
dd_train = dd_train.rename(columns=get_replace_dict('sub'))
dd_test = dd_test.rename(columns=get_replace_dict('sub'))

Wall time: 53.4 ms


In [17]:
dd_features.head(3)

,index_fea,id,buy_time_fea,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
0,0,2013026,2018-07-16,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,1,2014722,2018-10-15,36.690029,152.400888,448.069214,563.833202,463.841754,568.99821,-16.08618,...,-891.373846,-544.770792,-20.996269,48.369552,80.252276,-13.832889,-0.694428,-1.175933,-0.45614,0.0
2,2,2015199,2018-12-24,-67.019971,157.050888,-63.180786,178.103202,-68.598246,156.99821,3.51382,...,-977.373846,-613.770792,-12.996269,-37.630448,10829.252276,-25.832889,-0.694428,-12.175933,-0.45614,0.0


In [18]:
dd_train.head(3)

,index_sub,id,vas_id,buy_time_sub,target
0,0,540968,8.0,2018-09-17,0.0
1,1,1454121,4.0,2018-07-16,0.0
2,2,2458816,1.0,2018-08-13,0.0


In [19]:
dd_test.tail(3)

,index_sub,id,vas_id,buy_time_sub
71228,71228,1891350,2.0,2019-01-21
71229,71229,2437172,2.0,2019-01-21
71230,71230,988236,2.0,2019-01-21


### Объединение данных для train и features

In [20]:
%%time
merged_data_train = dd.merge(dd_train, dd_features, on=['id'], how='inner')

Wall time: 62.5 ms


In [21]:
merged_data_train.head(3)

,index_sub,id,vas_id,buy_time_sub,target,index_fea,buy_time_fea,0,1,2,...,243,244,245,246,247,248,249,250,251,252
0,140,4130548,2.0,2018-12-10,0.0,8832,2019-01-21,11.700029,17.790888,4.429214,...,-943.373846,-598.770792,-25.996269,-22.630448,-220.747724,-14.832889,-0.694428,-12.175933,-0.45614,1.0
1,842,540997,1.0,2018-11-05,0.0,11897,2018-12-24,-96.799971,-69.199112,-108.200786,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-24.832889,0.305572,-12.175933,-0.45614,0.0
2,843,540997,4.0,2018-11-19,1.0,11897,2018-12-24,-96.799971,-69.199112,-108.200786,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-24.832889,0.305572,-12.175933,-0.45614,0.0


In [ ]:
%%time
pd_merged_data_train = merged_data_train.compute()